In [14]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import numpy as np


# 产生随机分类数据集，10个特征， 2个类别
x, y = make_classification(n_samples=1000,n_features=10,n_classes=2)

rf = RandomForestClassifier()

print(np.mean(cross_val_score(rf, x, y, cv=20, scoring='roc_auc')))



#贝叶斯优化

#先要定义一个目标函数。
#比如此时，函数输入为随机森林的所有参数，输出为模型交叉验证5次的AUC均值，作为我们的目标函数。
#因为bayes_opt库只支持最大值，所以最后的输出如果是越小越好，那么需要在前面加上负号，以转为最大值。
def rf_cv(n_estimators, min_samples_split, max_features, max_depth):
    
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
        min_samples_split=int(min_samples_split),
        max_features=min(max_features, 0.999), # float
        max_depth=int(max_depth),random_state=2
        ),x, y, scoring='roc_auc', cv=5)

    return val.mean()



#建立贝叶斯优化对象
rf_bo = BayesianOptimization(
rf_cv,
{'n_estimators': (10, 250),
'min_samples_split': (2, 25),
'max_features': (0.1, 0.999),
'max_depth': (5, 15)}
)

#里面的第一个参数是我们的优化目标函数，第二个参数是我们所需要输入的超参数名称，以及其范围。超参数名称必须和目标函数的输入名称一一对应。
#开始优化：
rf_bo.maximize()


rf_bo.res["max"]

0.9478000000000002
|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9479   |  7.954    |  0.9316   |  11.75    |  205.5    |
|  2        |  0.9492   |  13.68    |  0.6646   |  20.23    |  173.9    |
|  3        |  0.9492   |  11.2     |  0.6155   |  10.76    |  172.7    |
|  4        |  0.9486   |  13.28    |  0.3569   |  19.36    |  64.3     |
|  5        |  0.9473   |  6.996    |  0.7189   |  4.522    |  201.1    |


TypeError: 'float' object is not subscriptable